# Import Libraries #

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
import cv2
import pickle
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras.preprocessing.image import img_to_array, load_img 
from sklearn.metrics import roc_curve, auc 
from tensorflow_model_optimization.sparsity import keras as sparsity

#import keras
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
#from keras.layers.normalization import BatchNormalization

from datetime import datetime

%load_ext tensorboard
import tensorboard
import tempfile
import zipfile
import os


In [ ]:
adam_opt = tf.keras.optimizers.Adam(learning_rate= 1e-3)
bce = tf.keras.losses.BinaryCrossentropy()

def check_binarized_auc(test_dset, model, X_test, y_test):
    '''Input:
            test_dset: test dataset to analyze performance of model on unseen data
            model: keras model trained previously
            
       Output:
            None: print out test AUC in function
    '''  
    y_pred = model.predict(X_test).ravel()
    fpr, tpr, threshold = roc_curve(y_test, y_pred)
    auc_model = auc(fpr, tpr)
    output = 'Test AUC: {}'
    print(output.format(auc_model))
    
    #sens and spec at Youden's index
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    sens = tpr[optimal_idx]
    spec = 1 - fpr[optimal_idx]
    print('Sensitivity: ' + str(sens))
    print('Specificity: ' + str(spec))
    
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='Model (area = {:.3f})'.format(auc_model))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    
def loss_plot(hist, epoch=50):
    plt.figure()
    plt.plot(range(epoch), hist.history['loss'], label='Training loss')
    plt.plot(range(epoch), hist.history['val_loss'], label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss Value')
    plt.legend()
    # END CODE #
    plt.show()

#functions to get nonzero weights 
def total_nonzero_weights(model):
    '''return total number of nonzero weights (total params)
    includes non-trainable params'''
    weights = model.get_weights()
    count_nz = []
    for i in range(len(weights)):
        count_nz.append(np.count_nonzero(weights[i]))
    return sum(count_nz)

def total_nonzero_trainable_weights(model):
    '''return total number of nonzero trainable weights'''
    w = []
    for v in model.trainable_variables:
        w.append(tf.math.count_nonzero(v))
    return sum(w)    

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("TF Version: ", tf.version.VERSION)

# Dictionary of images and labels #

In [ ]:
IMAGE_PATH =  '/home/djun36/project/ham10000_images/' 
LABEL_PATH = '/home/djun36/project/HAM10000_metadata.csv' 

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses', #Because it can become cancerous, it's usually removed as a precaution.
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

# Dictionary for binary Malignant and Benign labels. May want to change the terminology because of akiec 
lesion_referral_dict = {
    'nv': 'Benign',
    'mel': 'Malignant',
    'bkl': 'Benign',
    'bcc': 'Malignant',
    'akiec': 'Malignant',
    'vasc': 'Benign',
    'df': 'Benign'
}

# Process data #

In [ ]:
skin_df = pd.read_csv(LABEL_PATH)

# Creating New Columns for better readability

#skin_df['path'] = skin_df['image_id'].map(imageid_path_dict.get)
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get) 
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes

skin_df['referral'] = skin_df['dx'].map(lesion_referral_dict.get)

In [ ]:
# Now lets see the sample of tile_df to look on newly made columns
skin_df.head()

# Data Cleaning #

In [ ]:
skin_df.isnull().sum()

In [ ]:
skin_df['age'].fillna((skin_df['age'].mean()), inplace=True)

In [ ]:
skin_df.isnull().sum()

In [ ]:
print(skin_df.dtypes)

# Data Viz #

In [ ]:
# Plot to see distribution of 7 different classes of cell type
fig, ax1 = plt.subplots(1, 1, figsize= (10, 5))
skin_df['cell_type'].value_counts().plot(kind='bar', ax=ax1)

In [ ]:
skin_df['dx_type'].value_counts().plot(kind='bar')

In [ ]:
skin_df['localization'].value_counts().plot(kind='bar')

In [ ]:
skin_df['referral'].value_counts().plot(kind='bar')

print(skin_df['referral'].value_counts())
print('Benign %', 8061/10015, '\nMalignant %', 1954/10015)

# Balancing Dataset #
Randomly sample 1954 images from each class

In [ ]:
no_ref = skin_df.loc[skin_df['referral'] == 'Benign']['image_id'].values.tolist()
ref = skin_df.loc[skin_df['referral'] == 'Malignant']['image_id'].values.tolist()

random.seed(1)
no_refer_examples = random.sample(no_ref, 1954)
refer_examples = random.sample(ref, 1954)

all_files = no_refer_examples + refer_examples
random.shuffle(all_files)

# Resizing Images and Splitting Data #
Split data into train, tune, test with 0.6, 0.2, 0.2 split

In [ ]:
IMAGE_SIZE = (224, 224)

def load_data(all_files, IMAGE_SIZE):
    '''Input:
            all_files: list of image names to load in
            
       Output:
            X_train, X_tune, X_test: preprocessed images split in 0.6, 0.2, 0.2 respectively
            y_train, y_tune, y_test: binary labels split in 0.6, 0.2, 0.2 respectively
    '''  
    
    train_images = []
    train_labels = []
    
    #random.seed(1)
    #random.shuffle(all_files)
    labels_df = pd.read_csv(LABEL_PATH)

    for file in all_files:
        
        # FILL IN CODE HERE #
        
        img = load_img(IMAGE_PATH + file + '.jpg')
        array = img_to_array(img)
        resize = cv2.resize(array, IMAGE_SIZE, interpolation = cv2.INTER_LANCZOS4)
        norm_img = cv2.normalize(resize, None, 0, 1, norm_type=cv2.NORM_MINMAX)  #normalize image to 0 to 1
        
        if file in no_refer_examples:
            img_label = 0
        else:
            img_label = 1
         
        train_images.append(norm_img)
        train_labels.append(img_label)
        
        # END CODE #

    all_images = np.stack(train_images)
    all_labels = np.array(train_labels).flatten()
    
    total_data = len(all_images)
    num_train = round(total_data * 0.6)
    num_tune = round(total_data * 0.2)
    num_test = round(total_data * 0.2)
    
    # FILL IN CODE HERE #
    
    X_train = all_images[0:num_train]
    y_train = all_labels[0:num_train]
    X_tune = all_images[num_train: (num_train + num_tune)]
    y_tune = all_labels[num_train: (num_train + num_tune)]
    X_test = all_images[(num_train + num_tune):]
    y_test = all_labels[(num_train + num_tune):]
    
    # END CODE #

    return X_train, y_train, X_tune, y_tune, X_test, y_test

X_train, y_train, X_tune, y_tune, X_test, y_test = load_data(all_files, IMAGE_SIZE)

In [ ]:
#for iv3 data with size (299, 299)
iX_train, iy_train, iX_tune, iy_tune, iX_test, iy_test = load_data(all_files, (299, 299))

In [ ]:
#pickle.dump([X_train, y_train, X_tune, y_tune, X_test, y_test], open('ham10000_3.9k', 'wb'))
#pickle.dump([iX_train, iy_train, iX_tune, iy_tune, iX_test, iy_test], open('ham10000_3.9k_i', 'wb'))

X_train, y_train, X_tune, y_tune, X_test, y_test = pickle.load(open('ham10000_3.9k', 'rb'))
iX_train, iy_train, iX_tune, iy_tune, iX_test, iy_test = pickle.load(open('ham10000_3.9k_i', 'rb'))

In [ ]:
BATCH_SIZE = 16

train_dset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
tune_dset = tf.data.Dataset.from_tensor_slices((X_tune, y_tune)).batch(BATCH_SIZE)
test_dset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

itrain_dset = tf.data.Dataset.from_tensor_slices((iX_train, iy_train)).batch(BATCH_SIZE)
itune_dset = tf.data.Dataset.from_tensor_slices((iX_tune, iy_tune)).batch(BATCH_SIZE)
itest_dset = tf.data.Dataset.from_tensor_slices((iX_test, iy_test)).batch(BATCH_SIZE)

In [ ]:
for i in range(5):
    plt.imshow(X_train[i]) # Shows the image on a plot
    plt.title(y_train[i]) # Titles the image with the label
    plt.show()

# Create MN2 baseline model
We will be using a MobileNet (https://arxiv.org/abs/1704.04861) pre-trained on ImageNet as our base model and fine-tuning it on our dataset.

In [ ]:
mn2 = tf.keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet')

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(mn2.layers))

# Fine-tune from this layer onwards
mn2.trainable = True
fine_tune_at = 0

# Freeze all the layers before the `fine_tune_at` layer
for layer in mn2.layers[:fine_tune_at]:
  layer.trainable =  False

mn2.summary()

In [ ]:
# Build the model
h1 = tf.keras.layers.GlobalAveragePooling2D()(mn2.layers[-3].output) 
prediction = tf.keras.layers.Dense(units = 1, activation = 'sigmoid')(h1)
mn2_b = tf.keras.Model(inputs=mn2.input, outputs=prediction)

adam_opt = tf.keras.optimizers.Adam(learning_rate= 1e-3)
bce = tf.keras.losses.BinaryCrossentropy()

mn2_b.compile(optimizer= adam_opt,
              loss= bce,
              metrics=['accuracy'])

In [ ]:
mn2_b_hist = mn2_b.fit(train_dset, 
                 epochs=50,  
                 validation_data = tune_dset)

In [ ]:
#save model
mn2_b.save('mn2_b.h5')

#mn2_b = tf.keras.models.load_model('mn2_b.h5')

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

# Evaluate Model

In [ ]:
mn2_b.evaluate(test_dset)

Output with sigmoid, default lr, mn2_b: [1.1074739817757995, 0.8002561]

In [ ]:
check_binarized_auc(test_dset, mn2_b, X_test, y_test)
loss_plot(mn2_b_hist)

# Create Inception V3 baseline model

In [ ]:
iv3 = tf.keras.applications.InceptionV3(weights='imagenet')
#mobileNet = tf.keras.applications.MobileNet()
iv3.summary()

In [ ]:
# Let's take a look to see how many layers are in the base model

# Fine-tune from this layer onwards
iv3.trainable = True
fine_tune_at = 0

# Freeze all the layers before the `fine_tune_at` layer
for layer in iv3.layers[:fine_tune_at]:
  layer.trainable =  False

iv3.summary()
print("Number of layers in the base model: ", len(iv3.layers))
#print("trainable layers: ", len(iv3.trainable_variables))

In [ ]:
# Build the model
d1 = tf.keras.layers.Dropout(0.5)(iv3.layers[-3].output) 
h1 = tf.keras.layers.GlobalAveragePooling2D()(d1) #(iv3.layers[-3].output) 
prediction = tf.keras.layers.Dense(units = 1, activation = 'sigmoid')(h1)
iv3_b = tf.keras.Model(inputs=iv3.input, outputs=prediction)

adam_opt = tf.keras.optimizers.Adam(learning_rate= 1e-3)
bce = tf.keras.losses.BinaryCrossentropy()

iv3_b.compile(optimizer= adam_opt,
              loss= bce,
              metrics=['accuracy'])

In [ ]:
iv3_b_hist = iv3_b.fit(itrain_dset, 
                 epochs=50, 
                 #callbacks = [lr_scheduler], 
                 validation_data = itune_dset)

In [ ]:
#save model
#iv3_b.save('iv3_b.h5')

iv3_b = tf.keras.models.load_model('iv3_b.h5')

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

# Evaluate Model

In [ ]:
iv3_b.evaluate(itest_dset)

Output with sigmoid, default lr, iv3_b: [1.69085515989941, 0.7810499]

In [ ]:
check_binarized_auc(itest_dset, iv3_b, iX_test, iy_test)
loss_plot(iv3_b_hist)

# Create ResNet50v2 baseline model

In [ ]:
rn50 = tf.keras.applications.ResNet50V2(weights='imagenet')
rn50.summary()

In [ ]:
# Let's take a look to see how many layers are in the base model

# Fine-tune from this layer onwards
rn50.trainable = True
fine_tune_at = 0

# Freeze all the layers before the `fine_tune_at` layer
for layer in rn50.layers[:fine_tune_at]:
  layer.trainable =  False

rn50.summary()
print("Number of layers in the base model: ", len(rn50.layers))
print("trainable layers: ", len(rn50.trainable_variables))

In [ ]:
# Build the model
h1 = tf.keras.layers.GlobalAveragePooling2D()(rn50.layers[-3].output) 
prediction = tf.keras.layers.Dense(units = 1, activation = 'sigmoid')(h1)
rn50_b = tf.keras.Model(inputs=rn50.input, outputs=prediction)

adam_opt = tf.keras.optimizers.Adam(learning_rate= 1e-3)
bce = tf.keras.losses.BinaryCrossentropy()

rn50_b.summary()
print("Number of layers in the base model: ", len(rn50_b.layers))
print("trainable layers: ", len(rn50_b.trainable_variables))

'''rn50_b.compile(optimizer= adam_opt,
              loss= bce,
              metrics=['accuracy'])'''

In [ ]:
rn50_b_hist = rn50_b.fit(train_dset, 
                 epochs=75, 
                 #callbacks = [lr_scheduler], 
                 validation_data = tune_dset)

In [ ]:
#save model
rn50_b.save('rn50_b.h5')

#iv3_b = tf.keras.models.load_model('iv3_b.h5')

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

# Evaluate Model

In [ ]:
rn50_b.summary()

In [ ]:
rn50_b.evaluate(test_dset)

In [ ]:
check_binarized_auc(test_dset, rn50_b, X_test, y_test)
loss_plot(rn50_b_hist, 75)

# Create ResNet152v2 baseline model

In [ ]:
rn152 = tf.keras.applications.ResNet152V2(weights='imagenet')
#mobileNet = tf.keras.applications.MobileNet()
rn152.summary()

In [ ]:
#rn50
rn152 = tf.keras.applications.ResNet50V2(weights='imagenet')
rn152.summary()

In [ ]:
# Let's take a look to see how many layers are in the base model

# Fine-tune from this layer onwards
rn152.trainable = True
fine_tune_at = 0

# Freeze all the layers before the `fine_tune_at` layer
for layer in rn152.layers[:fine_tune_at]:
  layer.trainable =  False

rn152.summary()
print("Number of layers in the base model: ", len(rn152.layers))
#print("trainable layers: ", len(iv3.trainable_variables))

In [ ]:
# Build the model
h1 = tf.keras.layers.GlobalAveragePooling2D()(rn152.layers[-3].output) 
prediction = tf.keras.layers.Dense(units = 1, activation = 'sigmoid')(h1)
rn152_b = tf.keras.Model(inputs=rn152.input, outputs=prediction)

adam_opt = tf.keras.optimizers.Adam(learning_rate= 1e-2)
bce = tf.keras.losses.BinaryCrossentropy()

rn152_b.compile(optimizer= adam_opt,
              loss= bce,
              metrics=['accuracy'])

In [ ]:
rn152_b_hist = rn152_b.fit(train_dset, 
                 epochs=50, 
                 #callbacks = [lr_scheduler], 
                 validation_data = tune_dset)

In [ ]:
#save model
#rn152_b.save('rn50_b_l4.h5')

rn152_b = tf.keras.models.load_model('rn152_b.h5')

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

In [ ]:
rn152_b.summary()

In [ ]:
print("model:", total_nonzero_weights(rn152_b))

# Evaluate Model

In [ ]:
rn152_b.evaluate(test_dset)

Output with sigmoid, default lr, rn152_b: [2.0673098316302103, 0.73367476]

In [ ]:
check_binarized_auc(test_dset, rn152_b, X_test, y_test)
loss_plot(rn152_b_hist, 50)

# Evaluating Wrong Predictions

In [ ]:
mn2_b = tf.keras.models.load_model('mn2_b.h5')

In [ ]:
num_train = round(len(all_files) * 0.6)
num_tune = round(len(all_files) * 0.2)
y_test_images = all_files[(num_train + num_tune):]

In [ ]:
#y_pred = model.predict(X_test).ravel()
a = list(mn2_b.predict(X_test).ravel())
y_pred = []
for i in a:
    if i >= 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
cpy = []

for i in range(len(y_test_images)):
    a = skin_df.loc[skin_df['image_id']== y_test_images[i]].values.tolist()[0]
    a.append(y_test[i])
    a.append(y_pred[i])
    if y_test[i] == 1 and y_pred[i] == 1:
        a.append('TP')
    elif y_test[i] == 0 and y_pred[i] == 0:
        a.append('TN')
    elif y_test[i] == 1 and y_pred[i] == 0:
        a.append('FN')
    elif y_test[i] == 0 and y_pred[i] == 1:
        a.append('FP')
    cpy.append(a)

cpy_df = pd.DataFrame(cpy)

In [ ]:
# Plot to see distribution of 7 different classes of cell type
print(cpy_df[7].value_counts())
cpy_df[7].value_counts().plot(kind='bar')

In [ ]:
print(cpy_df.loc[cpy_df[12].isin(['FP','FN'])][7].value_counts())
cpy_df.loc[cpy_df[12].isin(['FP','FN'])][7].value_counts().plot(kind='bar')

In [ ]:
# Plot to see distribution of benigns & malignant
print(cpy_df[9].value_counts())
cpy_df[9].value_counts().plot(kind='bar')

In [ ]:
print(cpy_df.loc[cpy_df[12].isin(['FP','FN'])][9].value_counts())
cpy_df.loc[cpy_df[12].isin(['FP','FN'])][9].value_counts().plot(kind='bar')

In [ ]:
tf.math.confusion_matrix(
    y_test, y_pred, num_classes=2, weights=None, dtype=tf.dtypes.int32,
    name=None
)